# Task 2 Experimentation and uplift testing

In the task 2, we need to select the control stores and assessment of trial, finally we need to collate findings. 

In [1]:
#Import neccessary libraries
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np 

In [2]:
#Load the dataset
qvi=pd.read_csv('QVI_data.csv')
qvi.head()

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


In [3]:
#Check the infomation of the data
qvi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   LYLTY_CARD_NBR    264834 non-null  int64  
 1   DATE              264834 non-null  object 
 2   STORE_NBR         264834 non-null  int64  
 3   TXN_ID            264834 non-null  int64  
 4   PROD_NBR          264834 non-null  int64  
 5   PROD_NAME         264834 non-null  object 
 6   PROD_QTY          264834 non-null  int64  
 7   TOT_SALES         264834 non-null  float64
 8   PACK_SIZE         264834 non-null  int64  
 9   BRAND             264834 non-null  object 
 10  LIFESTAGE         264834 non-null  object 
 11  PREMIUM_CUSTOMER  264834 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 24.2+ MB


In [4]:
#Change the DATE datatype to Datetime
qvi.DATE=pd.to_datetime(qvi.DATE)

## Define Metrics 

The store trial was performed in stores 77,86 and 88. Firstly, we need to select the control stores which need us to explore the data and define metrics.
The trial period is start of February 2019 to the end of April 2019
We need to compare trial stores to control stores that are similar pre-trial, the similarity measurements are:
* Monthly overall sales revenue
* Monthly number of customers
* Monthly number of transactions per customer 

In [5]:
qvi['YEAR_MONTH']=qvi['DATE'].dt.strftime('%Y%m').astype('int')

In [6]:
#Define the metrics 
def store_monthly_metrics():
    store_monthly_group=qvi.groupby(['STORE_NBR','YEAR_MONTH'])
    tot_Sales=store_monthly_group['TOT_SALES'].sum()
    nCustomers=store_monthly_group['LYLTY_CARD_NBR'].nunique()
    nTxnPerCust=store_monthly_group.size()/nCustomers
    nChipsPerTxn=store_monthly_group['PROD_QTY'].sum()/store_monthly_group.size()
    avgPricePerUnit=tot_Sales/store_monthly_group['PROD_QTY'].sum()
    data=[tot_Sales,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit]
    headers=["tot_Sales","nCustomers","nTxnPerCust","nChipsPerTxn","avgPricePerUnit"]
    metrics=pd.concat(data,axis=1,keys=headers)
    return metrics

In [7]:
qvi_monthly_metrics =store_monthly_metrics().reset_index()
qvi_monthly_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3169 entries, 0 to 3168
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STORE_NBR        3169 non-null   int64  
 1   YEAR_MONTH       3169 non-null   int64  
 2   tot_Sales        3169 non-null   float64
 3   nCustomers       3169 non-null   int64  
 4   nTxnPerCust      3169 non-null   float64
 5   nChipsPerTxn     3169 non-null   float64
 6   avgPricePerUnit  3169 non-null   float64
dtypes: float64(4), int64(3)
memory usage: 173.4 KB


In [8]:
#Filter to the pre-trail period and stores with full observation periods
observ_counts = qvi_monthly_metrics["STORE_NBR"].value_counts()
full_observ_index = observ_counts[observ_counts == 12].index
full_observ = qvi_monthly_metrics[qvi_monthly_metrics["STORE_NBR"].isin(full_observ_index)]
pretrial_full_observ = full_observ[full_observ["YEAR_MONTH"] < 201902]
pretrial_full_observ.head(8)

,STORE_NBR,YEAR_MONTH,tot_Sales,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit
0,1,201807,206.9,49,1.061224,1.192308,3.337097
1,1,201808,176.1,42,1.023810,1.255814,3.261111
2,1,201809,278.8,59,1.050847,1.209677,3.717333
3,1,201810,188.1,44,1.022727,1.288889,3.243103
4,1,201811,192.6,46,1.021739,1.212766,3.378947
5,1,201812,189.6,42,1.119048,1.212766,3.326316
6,1,201901,154.8,35,1.028571,1.166667,3.685714
12,2,201807,150.8,39,1.051282,1.121951,3.278261


## Select control Stores

In [9]:
#Calculate the metrics corrlation between trial store and control store
def calCorrelation(metric, trialstore, inputTable=pretrial_full_observ):
    control_store_nbrs = inputTable[~inputTable["STORE_NBR"].isin([77, 86, 88])]["STORE_NBR"].unique()
    output= pd.DataFrame(columns=["Trial_Str", "Ctrl_Str", "Correlation"])
    trial_store = inputTable[inputTable["STORE_NBR"] == trialstore][metric].reset_index()
    for control in control_store_nbrs:
        control_store = inputTable[inputTable["STORE_NBR"] == control][metric].reset_index()
        concat_df = pd.DataFrame(columns = ["Trial_Str", "Ctrl_Str", "Correlation"])
        concat_df["Correlation"] = trial_store.corrwith(control_store, axis=1)
        concat_df["Trial_Str"] = trialstore
        concat_df["Ctrl_Str"] = control
        output =pd.concat([output, concat_df])
    return output

In [20]:
#Caculate the standardised magnitude difference between trial store and control 
def calcMagnitudeDistance(metric,trialstore,inputTable=pretrial_full_observ):
    control_store_nbrs = inputTable[~inputTable["STORE_NBR"].isin([77, 86, 88])]["STORE_NBR"].unique()
    output= pd.DataFrame()
    trial_store = inputTable[inputTable["STORE_NBR"] == trialstore][metric].reset_index(drop=True)
    for control in control_store_nbrs:
        control_store = inputTable[inputTable["STORE_NBR"] == control][metric].reset_index(drop=True)
        concat_df=abs(trial_store-control_store)
        concat_df["Trial_Str"] = trialstore
        concat_df["Ctrl_Str"] = control                 
        output =pd.concat([output, concat_df])
    for col in metric:
        output[col] = 1 - ((output[col] - output[col].min()) / (output[col].max() -output[col].min()))
    output["magnitude"] = output[metric].mean(axis=1)
    return output                  

In [23]:
trialstore=77
corr_table=calCorrelation(["tot_Sales", "nCustomers", "nTxnPerCust", "nChipsPerTxn", "avgPricePerUnit"],trialstore)
corr_table.head()

,Trial_Str,Ctrl_Str,Correlation
0,77,1,0.070544
1,77,1,0.027332
2,77,1,0.002472
3,77,1,-0.019991
4,77,1,0.030094


In [24]:
trialstore=77
corr_table=calcMagnitudeDistance(["tot_Sales", "nCustomers", "nTxnPerCust", "nChipsPerTxn", "avgPricePerUnit"],trialstore)
corr_table.head()

,tot_Sales,nCustomers,nTxnPerCust,nChipsPerTxn,avgPricePerUnit,Trial_Str,Ctrl_Str,magnitude
0,0.935431,0.980769,0.958035,0.497552,0.883569,77,1,0.851071
1,0.942972,0.951923,0.993823,0.571221,0.886328,77,1,0.869253
2,0.961503,0.836538,0.992126,0.428152,0.703027,77,1,0.784269
3,0.988221,0.932692,0.989514,0.880702,0.590528,77,1,0.876331
4,0.962149,0.951923,0.874566,0.535058,0.832481,77,1,0.831236


Based on the highest average composite score of total sales and number of customers,for trial store 77 we will chose the store 233 as control store, for trial store 86 we will chose the store 155 as control store, for trial store 88 we will chose the store 40 as control store.